In [ ]:
%pip install xgboost mlflow scikit-learn==1.4.0 pandas numpy matplotlib -qU

In [ ]:
from mlflow.tracking import MlflowClient

import json
import requests
import mlflow
import pandas as pd

In [ ]:
import warnings

warnings.simplefilter("ignore")


In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(experiment_name="heart-condition-classifier")


In [ ]:
file_url = "https://azuremlexampledata.blob.core.windows.net/data/heart-disease-uci/data/heart.csv"
df = pd.read_csv(file_url)


In [ ]:
df

In [ ]:
df["thal"] = df["thal"].astype("category").cat.codes


In [ ]:
df["thal"].unique()


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("target", axis=1), df["target"], test_size=0.3
)

In [ ]:
mlflow.xgboost.autolog()


In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")

In [ ]:
run = mlflow.start_run()


In [ ]:
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)



In [ ]:
y_pred = model.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score, recall_score


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)


In [ ]:
mlflow.end_run()



In [ ]:
run.info.run_id

In [ ]:
run = mlflow.get_run(run.info.run_id)


In [ ]:
pd.DataFrame(data=[run.data.params], index=["Value"]).T


In [ ]:
pd.DataFrame(data=[run.data.metrics], index=["Value"]).T


In [ ]:
client = mlflow.tracking.MlflowClient()
client.list_artifacts(run_id=run.info.run_id)


In [ ]:
file_path = mlflow.artifacts.download_artifacts(
    run_id=run.info.run_id, artifact_path="feature_importance_weight.png"
)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img



In [ ]:
image = img.imread(file_path)
plt.imshow(image)
plt.show()



In [ ]:
classifier = mlflow.xgboost.load_model(f"runs:/{run.info.run_id}/model")




In [ ]:
type(classifier)


In [ ]:
X_test.head()

In [ ]:
X_test.values[0]

In [ ]:
classifier.predict(X_test)